## General information about this notebook

This notebook series has been initiated by the Data Management Project (INF) within the TR-172 ["ArctiC Amplification: Climate Relevant Atmospheric and SurfaCe Processes, and Feedback Mechanisms" (AC)³](http://www.ac3-tr.de/) funded by the German Research Foundation (Deutsche Forschungsgemeinschaft, DFG)

Author(s) of this notebook: 
 - *Andreas Walbröl*, [*Institute for Geophysics and Meteorology, University of Cologne*](https://geomet.uni-koeln.de/), *Pohligstr. 3, 50969 Cologne*, *a.walbroel@uni-koeln.de*

Github repository: https://github.com/ac3-tr/ac3-notebooks

This notebook is licensed under the [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/ "CC-BY-4.0")

# Dataset description

**Title:**
- Integrated water vapour derived from the MiRAC-P microwave radiometer onboard the Polarstern during the MOSAiC expedition
- Temperature and humidity profiles, integrated water vapour and liquid water path derived from the HATPRO microwave radiometer onboard the Polarstern during the MOSAiC expedition

**Authors** 
- Walbröl, Andreas; Orlandi, Emiliano; Crewell, Susanne; Ebell, Kerstin
- Ebell, Kerstin; Walbröl, Andreas; Engelmann, Ronny; Griesche, Hannes; Radenz, Martin; Hofer, Julian; Althausen, Dietrich

**Description**
- The data set contains daily files of path integrated amount of water vapour (precipitable water, prw) derived from the MiRAC-P (or LHUMPRO-243-340) microwave radiometer (see Mech et al., 2019) onboard the Polarstern during cruise PS122 (MOSAiC expedition). The data covers the range October 2019 to October 2020. MiRAC-P measures atmospheric radiation in six double side band averaged G band (183.31 +/- 0.6 to 183.31 +/- 7.5 GHz) and two higher frequency (243 and 340 GHz) channels. The different sensitivity to humidity of the channels allow a derivation of the path integrated amount of water vapour (prw). A Neural Network retrieval has been used to derive prw. The uncertainty of prw is given as the variable's comment attribute and describes the expected standard error. Prw is provided for all available times so that it is up to the user to decide whether or not to use the values if quality flags are set.
- The data set contains daily files of path integrated amount of water vapour (precipitable water, prw) and cloud liquid (clwvi), as well as temperature (ta) and humidity (hua) profiles derived from the HATPRO microwave radiometer (see Rose et al., 2005) onboard the Polarstern during cruise PS122 (MOSAiC expedition). The data covers the range October 2019 to October 2020. HATPRO measures atmospheric radiation in seven K band (22.24 - 31.4 GHz) channels, which are sensitive to water vapour profiles, precipitable water and cloud liquid water. The seven V band (51.26 - 58 GHz) channels are sensitive to atmospheric temperature and therefore applicable to derive the temperature profiles. Multivariate regression retrievals have been used to derive the atmospheric variables (see Löhnert and Crewell, 2003, Löhnert and Maier, 2012 and Nomokonova et al. 2019). Temperature profiles have been retrieved from both zenith (mwr00) and boundary layer (mwrBL00) modes. The uncertainties of the variables describe the expected standard error (prw_err, clwvi_err, ...). Each retrieved variable is provided for all available times so that it is up to the user to decide whether or not to use the values if quality flags are set.

**Year**       
- 2022
- 2022

**Institutes**  
- Institute for Geophysics and Meteorology, University of Cologne
- Institute for Geophysics and Meteorology, University of Cologne

**DOI**        
- [https://doi.org/10.1594/PANGAEA.941470](https://doi.org/10.1594/PANGAEA.941470)
- [https://doi.org/10.1594/PANGAEA.941389](https://doi.org/10.1594/PANGAEA.941389)

**License**    [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/ "CC-BY-4.0")

## Contents of this notebook

This notebook provides an example how to import and visualize integrated water vapour (prw), liquid water path (clwvi), temperature profiles (ta) and humidity profiles (hua) derived from microwave radiometer measurements during the MOSAiC expedition (POLARSTERN cruise PS122). Be aware that if you download all variables MOSAiC year, you need roughly 25 GB of disk space. The temperature and humidity profile data require most space. 

In this example, the integrated water vapour (prw) and liquid water path (clwvi) will be plotted for the whole available period while temperature and humidity profiles are only shown for a few days because of memory usage.


## Import relevant modules

The following packages are needed for this notebook: _xarray_, _numpy_, _matplotlib_

In [ ]:
import xarray as xr
import numpy as np
import glob
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

## Pre-processing of the imported data

Download the data you would like to plot (i.e., integrated water vapour (prw), liquid water path (clwvi), ...) and save them in the same folder as your notebook.

In [ ]:
def find_files_daterange(
    all_files, 
    date_start_dt, 
    date_end_dt,
    idx):

    """
    Filter from a given set of files the correct ones within the date range
    date_start_dt - date_end_dt (including start and end date).

    Parameters:
    -----------
    all_files : list of str
        List of str that includes all the files.
    date_start_dt : datetime object
        Start date as a datetime object.
    date_end_dt : datetime object
        End date as a datetime object.
    idx : list of int
        List of int where the one entry specifies the start and the second one
        the end of the date string in any all_files item (i.e., [-17,-9]).
    """

    files = list()
    for pot_file in all_files:
        # check if file is within our date range:
        file_dt = dt.datetime.strptime(pot_file[idx[0]:idx[1]], "%Y%m%d")
        if (file_dt >= date_start_dt) & (file_dt <= date_end_dt):
            files.append(pot_file)

    return files


path_data = "./"
date_start = "2020-04-19"    # start date of the temperature and humidity profiles, in yyyy-mm-dd
date_end = "2020-04-20"      # end date of the temperature and humidity profiles, in yyyy-mm-dd

date_start_dt = dt.datetime.strptime(date_start, "%Y-%m-%d")
date_end_dt = dt.datetime.strptime(date_end, "%Y-%m-%d")


# identify files:
all_files = {'prw': sorted(glob.glob(path_data + "ioppol_tro_mwr00_l2_prw_v01_*.nc")),
             'clwvi': sorted(glob.glob(path_data + "ioppol_tro_mwr00_l2_clwvi_v01_*.nc")),
             'hua': sorted(glob.glob(path_data + "ioppol_tro_mwr00_l2_hua_v01_*.nc")),
             'ta': sorted(glob.glob(path_data + "ioppol_tro_mwr00_l2_ta_v01_*.nc")),
             'ta_bl': sorted(glob.glob(path_data + "ioppol_tro_mwrBL00_l2_ta_v01_*.nc"))
            }
all_files_mir = {'prw': sorted(glob.glob(path_data + "MOSAiC_uoc_lhumpro-243-340_l2_prw_v01*.nc"))}

# limit temperature and humidity profiles to the desired date range:
for key in ['hua', 'ta', 'ta_bl']:
    all_files[key] = find_files_daterange(all_files[key], date_start_dt, date_end_dt, [-17,-9])


# Import retrieved data:
HATPRO_DS = dict()    # dictionary containing xarray datasets:
for key in all_files.keys():
    HATPRO_DS[key] = xr.open_mfdataset(all_files[key], concat_dim='time', combine='nested')
MIRAC_DS = {'prw': xr.open_mfdataset(all_files_mir['prw'], concat_dim='time', combine='nested')}


# Filter flagged values (flag > 0):
for key in HATPRO_DS.keys():
    ok_idx = np.where((HATPRO_DS[key].flag.values == 0) | (np.isnan(HATPRO_DS[key].flag.values)))[0]
    HATPRO_DS[key] = HATPRO_DS[key].isel(time=ok_idx)

ok_idx = np.where((MIRAC_DS['prw'].flag.values == 0) | (np.isnan(MIRAC_DS['prw'].flag.values)))[0]
MIRAC_DS['prw'] = MIRAC_DS['prw'].isel(time=ok_idx)

## Plotting example

**Time series of integrated water vapour and liquid water path:**

In [ ]:
f1 = plt.figure(figsize=(10,15))
ax_iwv = plt.subplot2grid((2,1), (0,0))    # IWV (prw)
ax_lwp = plt.subplot2grid((2,1), (1,0))    # LWP (clwvi)

# axis limits:
iwv_lims = [0, 35]    # in kg m-2
lwp_lims = [0, 750]   # in g m-2

# plot iwv and lwp:
ax_iwv.plot(HATPRO_DS['prw'].time.values, HATPRO_DS['prw'].prw.values, color='blue', label='HATPRO')
ax_iwv.plot(MIRAC_DS['prw'].time.values, MIRAC_DS['prw'].prw.values, color='cyan', label='MiRAC-P')
ax_lwp.plot(HATPRO_DS['clwvi'].time.values, HATPRO_DS['clwvi'].clwvi.values*1000.0, color='k')

# legend:
ax_iwv.legend()

# labels:
ax_iwv.set_ylabel("Integrated water vapour ($\mathrm{kg}\,\mathrm{m}^{-2}$)")
ax_lwp.set_xlabel("Time")
ax_lwp.set_ylabel("Liquid water path ($\mathrm{g}\,\mathrm{m}^{-2}$)")

**Time-height cross section of temperature and humidity profiles:**

In [ ]:
# colormaps and levels:
rho_v_cmap = mpl.cm.gist_earth
temp_cmap = mpl.cm.nipy_spectral
rho_v_levels = np.arange(0, 5.51, 0.2)    # in g m-3
temp_levels = np.arange(200, 285.001, 2)    # in K


fig1 = plt.figure(figsize=(10,15))
ax_hua = plt.subplot2grid((3,1), (0,0))    # humidity profiles
ax_ta = plt.subplot2grid((3,1), (1,0))    # temperature profiles from zenith observations
ax_ta_bl = plt.subplot2grid((3,1), (2,0))    # temperature profiles from boundary layer scan

# axis limits:
height_lims = [0, 8000]    # in m


# plot data:
xv, yv = np.meshgrid(HATPRO_DS['hua'].height.values, HATPRO_DS['hua'].time.values)
rho_v_hat_curtain = ax_hua.contourf(yv, xv, 1000*HATPRO_DS['hua'].hua.values, levels=rho_v_levels,
                                        cmap=rho_v_cmap)
xv, yv = np.meshgrid(HATPRO_DS['ta'].height.values, HATPRO_DS['ta'].time.values)
temp_hat_curtain = ax_ta.contourf(yv, xv, HATPRO_DS['ta'].ta.values, levels=temp_levels,
                                      cmap=temp_cmap)
xv, yv = np.meshgrid(HATPRO_DS['ta_bl'].height.values, HATPRO_DS['ta_bl'].time.values)
temp_bl_hat_curtain = ax_ta_bl.contourf(yv, xv, HATPRO_DS['ta_bl'].ta.values, levels=temp_levels,
                                      cmap=temp_cmap)


# some subplot identifiers:
ax_hua.text(0.02, 0.95, "HATPRO humidity profile", color=(1,1,1), fontweight='bold', ha='left', va='top', transform=ax_hua.transAxes)
ax_ta.text(0.02, 0.95, "HATPRO temperature profile (zenith)", color=(1,1,1), fontweight='bold', ha='left', va='top', transform=ax_ta.transAxes)
ax_ta_bl.text(0.02, 0.95, "HATPRO temperature profile (BL mode)", color=(1,1,1), fontweight='bold', ha='left', va='top', transform=ax_ta_bl.transAxes)

# colorbars:
cb_hua = fig1.colorbar(mappable=rho_v_hat_curtain, ax=ax_hua, use_gridspec=True, extend='max', orientation='vertical', fraction=0.09, pad=0.01, shrink=0.9)
cb_hua.set_label(label="$\\rho_v$ ($\mathrm{g}\,\mathrm{m}^{-3}$)")
cb_ta = fig1.colorbar(mappable=temp_hat_curtain, ax=ax_ta, use_gridspec=True, extend='max', orientation='vertical', fraction=0.09, pad=0.01, shrink=0.9)
cb_ta.set_label(label="T (K)")
cb_ta_bl = fig1.colorbar(mappable=temp_bl_hat_curtain, ax=ax_ta_bl, use_gridspec=True, extend='max', orientation='vertical', fraction=0.09, pad=0.01, shrink=0.9)
cb_ta_bl.set_label(label="T (K)")